In [ ]:
import requests
import pandas as pd 
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from functions import get_all_pages, transform_staff, upload_to_postgres, upload_to_postgres

load_dotenv()
partner_token = os.getenv('PARTNER_TOKEN')
login = os.getenv('LOGIN')
password = os.getenv('PASSWORD')
company_id = os.getenv('COMPANY_ID')
partner_id = os.getenv('PARTNER_ID')

pd.options.display.max_rows = 200
pd.options.display.max_columns = 80

### Авторизация и получение данных о пользователе

In [13]:
url = 'https://api.yclients.com/api/v1/auth'

headers = {
    "Authorization": f"Bearer {partner_token}",
    "Accept": "application/vnd.yclients.v2+json"
}
payload = {
    "login": login,
    "password": password
}

response = requests.post(url, headers=headers, json=payload)
user_token = response.json()['data']['user_token'] # получаем юзер токен
print(response.status_code)

201


### Устанавливаем заголовки для последующих запросов

In [14]:
headers = {
    "Accept": "application/vnd.yclients.v2+json",
    "Content-Type" : "application/json",
    "Authorization": f"Bearer {partner_token}, User {user_token}"
}

### Получение данных по персоналу

  * id	-	Идентификатор сотрудника
  * name	-	Имя сотрудника
  * specialization	-	Специализация сотрудника
  * position.title	-	Должность сотрудника
  * weight	-	Вес сотрудника. При выводе сотрудники сортируются по весу, сначала более тяжелые
  * rating	-	Ретинг сотрудника
  * avatar	-	Путь к файлу аватарки сотрудника
  * avatar_big	-	Путь к файлу аватарки сотрудника в более высоком разрешении
  * information	-	Дополнительная информация о сотруднике (HTML формат)
  * hidden	-	1 - скрыт от онлайн записей, 0 - не скрыт
  * fired	-	1 - уволен, 0 - не уволен
  * dismissal_date - дата увольнения
  * dismissal_reason - причина увольнения
  * schedule_till - дата, до которой действует текущее расписание сотрудника
  * has_schedule - флаг, указывающий, есть ли у сотрудника настроенное расписание
  * user.phone - номер телефона
  * user.email - почта

In [15]:
url = f"https://api.yclients.com/api/v1/company/{company_id}/staff/"
response = requests.get(url, headers=headers)

staff = pd.json_normalize(response.json()['data'])[[
    'id', 'name', 'specialization', 'position.title', 'weight', 'rating', 'avatar', 
    'avatar_big', 'information', 'hidden', 'fired', 'dismissal_date', 'dismissal_reason', 
    'schedule_till', 'has_schedule', 'user.phone', 'user.email'
    ]]
# staff

### Получение данных по категориям услуг компании

  * id	- Идентификатор категории
  * title	- Название категории
  * weight	- Вес категории (используется для сортировки категорий при отображении)
  * staff	- Список ID сотрудников, оказывающих услугу

In [16]:
url = f"https://api.yclients.com/api/v1/company/{company_id}/service_categories/"
response = requests.get(url, headers=headers)

service_categories = pd.json_normalize(response.json()['data'])[[
    'id', 'category_id', 'salon_service_id', 'title', 'weight', 'staff'
    ]]
# service_categories

### Получение данных по услугам компании

  * booking_title	-	Название категории
  * service_type	-	1 - доступна для онлайн записи, 0 - не доступна
  * schedule_template_type - тип расписания
    * 0 (или null) — Расписание не настроено (нет шаблона).
    * 1 — Фиксированное расписание (одинаковое на каждую неделю).
    * 2 — Скользящий график (например, 2/2, 3/1 — повторяющиеся смены).
    * 3 — Индивидуальное расписание (настроено вручную для конкретных дней)
  * Статус онлайн оплаты: 0 — отключена, 1 — включена, 2 — Ограниченная онлайн-оплата
  * id	- Идентификатор услуги
  * category_id	-	Идентификатор категории, в которой состоит услуга
  * weight	-	Вес категории (используется для сортировки категорий при отображении)
  * staff	-	Список сотрудников, оказывающих услугу и длительность сеанса
  * duration	-	Длительность услуги, по умолчанию равна 3600 секундам

In [17]:
url = f"https://api.yclients.com/api/v1/company/{company_id}/services/"
response = requests.get(url, headers=headers)

services = pd.json_normalize(response.json()['data'])[[
    'booking_title', 'service_type', 'schedule_template_type', 'online_invoicing_status',
    'price_prepaid_percent', 'id', 'salon_service_id', 'category_id', 'weight', 'staff', 'duration'
]]
services['staff'] = services['staff'].apply(transform_staff)
# services.head()

### Получаем данные по клиентам

In [ ]:
url = f"https://api.yclients.com/api/v1/company/{company_id}/clients/search"

body = {
    "fields": ["id", "name", "surname", "phone", "email"]
}
clients = pd.DataFrame(get_all_pages(url, headers, method='POST', body=body))
# clients

### Получаем данные по записям

  * id - идентификатор записи
  * staff_id - идентификатор сотрудника
  * services - инфо по услуге
  * goods_transactions - инфо по сопутствующим товарам
  * client - идентификатор клиента
  * date - дата записи
  * create_date - дата создания записи
  * attendance - Общий статус посещения
    * 0 — Не обработан (запись создана, но клиент не отмечен).
    * 1 — Посещение завершено (клиент пришёл, услуги оказаны).
    * 2 — Клиент не пришёл (но запись не отменена).
    * 3 — Запись отменена.
  * length - длительность услуги
  * visit_id - идентификатор визита, не то же самое что идентификатор записи
  * paid_full - идентификатор полной оплаты
  * payment_status - статус оплаты
    * 0	Не оплачено	Клиент не произвёл оплату.
    * 1	Оплачено частично	Внесена часть суммы (например, предоплата).
    * 2	Оплачено полностью	Услуга/товар оплачены полностью.
    * 3	Возврат	Средства возвращены клиенту.
    * 4	Ожидает подтверждения	Платёж в обработке (например, банковский перевод).

In [ ]:
url = f"https://api.yclients.com/api/v1/records/{company_id}"

records = pd.DataFrame(get_all_pages(url, headers))[[
    'id', 'staff_id', 'services', 'goods_transactions', 'client', 'date',
    'create_date', 'attendance', 'length', 'visit_id', 'paid_full', 'payment_status'
]]
records.loc[records['client'].notnull(), 'client'] \
    = records.loc[records['client'].notnull(), 'client'].apply(lambda x: x.get('id'))
# records

### Получаем список товаров
  * title	-	Наименование товара
  * category - Категория
  * category_id - Идентификатор категории
  * good_id	-	Идентификатор товара
  * cost	-	Цена продажи
  * unit_short_title	-	Единица измерения для продажи
  * actual_cost	-	Себестоимость
  * last_change_date	string	Дата последнего изменения сущности

In [39]:
url = f"https://api.yclients.com/api/v1/goods/{company_id}/"

goods = pd.DataFrame(get_all_pages(url, headers))[[
    'title', 'category', 'category_id', 'good_id', 'cost', 
    'unit_short_title', 'actual_cost','last_change_date'
]]
# goods 

Загрузка : 347it [00:05, 65.43it/s]


### Получаем данные по транакциям продуктов 
# ОШИБКА

In [ ]:
# url = "https://api.yclients.com/api/v1/company/{company_id}/goods/transactions"

# goods_transactions = pd.DataFrame(get_all_pages(url, headers))
# goods_transactions

Загрузка transactions: 0it [00:00, ?it/s]

Ошибка запроса: 404 {"success":false,"data":null,"meta":{"message":"Произошла ошибка"}}


""


### Получаем расписание сотрудников

In [ ]:
# url = f"https://api.yclients.com/api/v1/company/{company_id}/staff/schedule"
# params = {
#     "start_date":"2024-01-01", 
#     "end_date" :"2025-05-30"
# }
# response = requests.get(url, headers=headers, params=params)
# response.json()['data']

### Задаем параметры для поключения к БД

In [ ]:
# === Данные подключения к PostgreSQL ===
db_user = 'user'
db_password = 'password'
db_name = 'Monada'
db_host = 'localhost'
db_port = '5433'

# Строка подключения
connection_string = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)

### Создаем функции получения данных каждой сущности

In [ ]:
def get_goods_transactions():
    url = f"https://api.yclients.com/api/v1/company/{company_id}/goods/transactions"
    df = pd.DataFrame(get_all_pages(url, headers, method='GET'))
    return df

def get_finance_pay_methods():
    url = f"https://api.yclients.com/api/v1/finance/pay_methods/{company_id}"
    df = pd.DataFrame(get_all_pages(url, headers, method='GET'))
    return pd.json_normalize(df)

def get_finance_operations():
    url = f"https://api.yclients.com/api/v1/finance/operations/{company_id}"
    df = pd.DataFrame(get_all_pages(url, headers, method='GET'))
    return pd.json_normalize(df)

def get_schedules():
    url = f"https://api.yclients.com/api/v1/company/{company_id}/schedules"
    df = pd.DataFrame(get_all_pages(url, headers, method='GET'))
    return pd.json_normalize(df)


### Загружаем данные в БД

In [ ]:
# === Основной процесс ===

print("🚀 Начинаем полную загрузку данных из YClients и запись в PostgreSQL...\n")

entities = {
    "clients": clients,
    "records": records,
    "staff": staff,
    "service_categories": service_categories,
    "services" : services,
    "goods": goods
    # "goods_transactions": goods_transactions,
    # "finance_operations": finance_operations,
    # "schedules": schedules
}

for table_name, df in entities.items():
    print(f"\n=== {table_name.upper()} ===")
    upload_to_postgres(df, table_name)

print("\n🎉 Все данные успешно загружены в БД!")